In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install "dask[dataframe]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc
import matplotlib.pyplot as plt
from sklearn import preprocessing
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_samples, silhouette_score
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation

In [ ]:
#path2 = ('/content/drive/MyDrive/Data/mock.nc')
#path2 = ('/content/drive/MyDrive/Private/Image_Similarity/mock1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v2.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v2.1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v3.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v3.1.nc')
#path2 = ('/content/drive/MyDrive/Data/mock_v4.nc')
path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_daily_smalldomain.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly.nc')
#path2 = ('/content/drive/MyDrive/Data/ERA5_meteo_sfc_2021_hourly_smalldomain.nc')
data = xr.open_dataset(path2, decode_times=False) #To view the date as integers of 0, 1, 2,....
#data = xr.open_dataset(path2)# decode_times=False) #To view the date as integers of 0, 1, 2,....
#data5 = xr.open_dataset(path2) # To view time in datetime format
data

<xarray.Dataset>
Dimensions:    (time: 365, longitude: 41, latitude: 41)
Coordinates:
  * time       (time) float64 0.0 1.0 2.0 3.0 4.0 ... 361.0 362.0 363.0 364.0
  * longitude  (longitude) float32 -33.0 -32.75 -32.5 ... -23.5 -23.25 -23.0
  * latitude   (latitude) float32 43.0 42.75 42.5 42.25 ... 33.5 33.25 33.0
Data variables:
    sp         (time, latitude, longitude) float64 ...
    sst        (time, latitude, longitude) float64 ...
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    sshf       (time, latitude, longitude) float64 ...
    slhf       (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
Attributes:
    title:          Daily variables from ERA5 hourly data
    history:        Wed Mar 23 12:01:11 2022: ncrcat ERA5_meteo_sfc_2021_01-0...
    Input dataset:  /global/cscratch1/sd/xzheng/ENA/rawdata/ERA5_meteo_sfc_20...
    NCO:            netCDF Operators version 5.0.1 (Homepage = http://nco.sf....

In [ ]:
# data = data.drop_vars('sst')
# data = data.drop_vars("slhf")
# data = data.drop_vars("sp")
# data = data.drop_vars("sshf")
# data = data.drop_vars('u10')
# data = data.drop_vars('v10')
# data

In [ ]:
def datatransformation(data):
        dask_df = data.to_dask_dataframe(dim_order=None, set_index=False)
        pd_df = dask_df.compute()

        for i in pd_df.columns:
          if pd_df[i].isna().sum() > 0:
            pd_df[i].fillna(value=pd_df[i].mean(), inplace=True)
        
        #col = 'time','lat','lon'
        col = 'time','latitude','longitude'
        fin_df = pd_df.loc[:, ~pd_df.columns.isin(col)]

        trans_data = pd.DataFrame()
        for j in fin_df.columns:
          for i in range(0,pd_df.shape[0]):
              c=(j + '(' + str(pd_df.latitude[i])+','+str(pd_df.longitude[i]) + ')')
              trans_data.loc[pd_df.time[i], c] = pd_df[j][i]

        return trans_data

def datanormalization(input):
  x = input.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  trans_data = pd.DataFrame(x_scaled, columns=input.columns, index=input.index)
        
  return trans_data

In [ ]:
#df = datatransformation(data)

In [ ]:
#df.to_csv("/content/drive/MyDrive/Private/Image_Similarity/df_daily_trans_PCA_11.csv")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Private/Image_Similarity/df_daily_trans_norm.csv")

In [ ]:
df = df.iloc[:,1:]

In [ ]:
df

,"sp(43.0,-33.0)","sp(42.75,-33.0)","sp(42.5,-33.0)","sp(42.25,-33.0)","sp(42.0,-33.0)","sp(41.75,-33.0)","sp(41.5,-33.0)","sp(41.25,-33.0)","sp(41.0,-33.0)","sp(40.75,-33.0)",...,"t2m(35.25,-23.0)","t2m(35.0,-23.0)","t2m(34.75,-23.0)","t2m(34.5,-23.0)","t2m(34.25,-23.0)","t2m(34.0,-23.0)","t2m(33.75,-23.0)","t2m(33.5,-23.0)","t2m(33.25,-23.0)","t2m(33.0,-23.0)"
0,0.769017,0.763551,0.758399,0.754214,0.749653,0.744597,0.739195,0.733298,0.726633,0.719377,...,0.307300,0.312956,0.318721,0.328563,0.334153,0.337154,0.340863,0.343122,0.338263,0.331969
1,0.550508,0.539280,0.528615,0.520168,0.510767,0.500749,0.491037,0.481886,0.472325,0.462340,...,0.366677,0.364151,0.363580,0.368759,0.371705,0.372464,0.370914,0.367119,0.358756,0.353140
2,0.406872,0.391876,0.377439,0.365852,0.353486,0.340685,0.327491,0.314235,0.300378,0.286403,...,0.382203,0.384572,0.381714,0.380983,0.379379,0.377956,0.377146,0.375601,0.372759,0.375038
3,0.305465,0.282306,0.260121,0.241803,0.223679,0.206865,0.191111,0.176584,0.162586,0.149556,...,0.391246,0.383770,0.374375,0.375442,0.382104,0.386662,0.388110,0.389274,0.388262,0.391018
4,0.421539,0.398652,0.376436,0.357109,0.337143,0.317032,0.296732,0.276819,0.256749,0.235928,...,0.339481,0.371150,0.384408,0.382828,0.380475,0.379120,0.382806,0.388641,0.391018,0.390756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,0.052561,0.047201,0.044078,0.046307,0.048426,0.051497,0.055427,0.060420,0.066650,0.073432,...,0.535796,0.539179,0.545987,0.558745,0.566072,0.565328,0.562305,0.556372,0.547556,0.544027
361,0.000000,0.000000,0.001187,0.007059,0.011975,0.016685,0.020693,0.024733,0.028452,0.031877,...,0.532577,0.529579,0.530462,0.536012,0.536755,0.532624,0.527203,0.521547,0.513866,0.509431
362,0.062689,0.061612,0.062288,0.067914,0.073401,0.079448,0.085514,0.091616,0.097813,0.104082,...,0.536404,0.543597,0.551860,0.563950,0.569878,0.569516,0.566563,0.562419,0.556039,0.552496
363,0.013697,0.005919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500904,0.507099,0.516941,0.534650,0.546183,0.551323,0.552699,0.551484,0.549971,0.547362


###Implementing Agglomerative Hierarchical clustering

In [ ]:
pca = PCA(n_components=11)
principalComponents = pca.fit_transform(df)
trans_data = pd.DataFrame(data = principalComponents
             , columns = ['PC_1', 'PC_2', 'PC_3','PC_4', 'PC_5', 'PC_6','PC_7', 'PC_8', 'PC_9','PC_10', 'PC_11'])

In [ ]:
trans_data

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11
0,14.328984,5.606987,-5.204885,12.194085,-0.825694,-4.529356,0.694313,0.904996,1.887883,-0.472180,0.592550
1,14.957689,-1.524946,-2.361496,18.392509,0.186625,-4.072603,-1.756697,3.100150,3.021399,2.228468,2.071070
2,15.526778,-8.950955,1.690469,21.770105,-4.871350,-2.274153,-1.098372,4.251684,2.159362,4.446972,1.603619
3,16.389001,-10.857179,3.874567,17.919476,-7.860215,-1.182745,-4.856464,3.571120,-3.837224,2.266003,-1.042942
4,16.727043,-6.238771,7.228452,21.994348,-7.489061,-5.323607,-5.212077,4.896106,-1.469530,1.188184,-0.076807
...,...,...,...,...,...,...,...,...,...,...,...
360,10.058760,-20.981553,-2.595284,-1.207836,4.150197,2.409976,-4.866501,1.233953,4.593244,0.712761,-1.230011
361,10.803433,-25.945878,-5.390507,2.160048,4.050436,2.240109,-0.614656,0.830943,6.883619,0.011428,-0.276090
362,14.400948,-18.954545,8.912374,2.606090,4.530542,2.369073,2.829505,3.111842,2.040401,-3.150608,3.045133
363,16.296198,-20.407814,15.441413,6.122403,5.511515,2.673390,-0.928345,-0.055634,1.736784,-0.469029,3.703328


In [ ]:
model = AffinityPropagation(damping=0.9, max_iter=200, convergence_iter=15, copy=True, preference=None, affinity='euclidean', verbose=False, random_state=None)
# fit the model
y_model = model.fit(trans_data)

labels = y_model.labels_

df1 = pd.DataFrame(trans_data)
df1['Cluster'] = labels
df1['Cluster'].value_counts()


10    35
17    31
15    27
7     23
20    22
12    22
14    21
13    19
1     17
5     16
9     15
19    15
18    14
16    12
4     11
11    10
6      9
8      9
24     8
2      8
21     8
3      5
23     4
22     3
0      1
Name: Cluster, dtype: int64

In [ ]:
classification = labels
classification

array([ 7,  9,  9,  9,  9,  0,  8,  8,  7,  7,  7,  1, 10, 10, 10, 10,  1,
        1,  5,  1,  5, 10,  1,  1,  1,  1,  4,  1,  4,  1,  1,  4,  2,  3,
        3,  8,  6,  5,  5,  5,  4,  4,  4,  4,  2,  2,  5,  5,  4,  2,  3,
        5,  4,  2,  8,  7,  6,  7,  7,  5,  4,  5,  5,  7,  6,  6,  6,  6,
        6,  7,  7,  7,  1, 10,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        6,  6,  4,  9,  9,  7,  9,  9,  9,  9,  9,  9,  5,  9,  9,  9,  1,
        5, 10, 10, 10, 10, 10, 10, 10,  1, 24,  3,  3,  5,  7,  7,  7,  8,
        7, 10, 10, 10, 10, 10, 10,  1,  5,  5, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 12, 11, 11, 12, 12, 10, 10, 11, 12, 12, 12, 11, 11, 11,
       12, 12, 12, 12, 10, 10, 10, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 20, 20, 20, 20, 20, 20, 19, 14, 14, 14, 14, 14,
       13, 13, 13, 13, 15, 15, 18, 20, 14, 14, 14, 14, 14, 14, 14, 14, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 15, 15,
       17, 17, 17, 17, 17

In [ ]:
#For Spatial Correlation coefficient computation
'''
   Input parameters: 
   X : An array, A 2-D array containing multiple variables and observations. 
   Y : An array, A 2-D array containing multiple variables and observations. 
   Each row of x represents a variable, and each column a single observation of all those variables.

   Returns: Pearson product-moment correlation coefficients.:A scaler quantity [-1, 1] 
   where -1 means the inputs are inversly correlated(opposite movements) 
   and 1 means directly correlated (unidirectional parallel movement) 
   while 0 means no correlation.
   
'''


def pearson_PM(x, y):

  #convert format from netcdf to np array
    #x_form = x.to_numpy()
    #y_form = y.to_numpy()

    #Flatten/transform from 2d to 1d
    X_flat = x.flatten()
    Y_flat = y.flatten()

    #Compute correlation matrix
    corr_mat = np.corrcoef(X_flat, Y_flat)

    #Return entry [0,1]
    return corr_mat[0,1]

In [ ]:
# # Converting the normalized data array into a pandas dataframe
# trans_data = datatransformation(data)
# nor_data = np.array(datanormalization(trans_data)) # This is just a trial
# trans_data = pd.DataFrame(nor_data, columns=trans_data.columns, index=trans_data.index)
# # Adding class centers and cluster numbers as columns to the dataframe
# #trans_data['Class_Center'] = class_centers
# trans_data['Cluster'] = classification
# # Rearranging the columns in the dataframe
# #trans_data = trans_data[['Class_Center', 'Cluster'] + [c for c in trans_data if c not in ['Class_Center', 'Cluster']]]
# #trans_data1 = trans_data
# #trans_data = trans_data.reset_index()
# trans_data

In [ ]:
#trans_data.to_csv("/content/drive/MyDrive/Private/Image_Similarity/trans_data_daily.csv")

In [ ]:
trans_data = pd.read_csv("/content/drive/MyDrive/Private/Image_Similarity/df_daily_trans_norm.csv")

In [ ]:
trans_data = trans_data.iloc[:,1:]

In [ ]:
trans_data['Cluster'] = classification

In [ ]:
trans_data.to_csv("/content/drive/MyDrive/Private/Image_Similarity/for_silh_PCA_11_Norm.csv")

In [ ]:
trans_data

,"sp(43.0,-33.0)","sp(42.75,-33.0)","sp(42.5,-33.0)","sp(42.25,-33.0)","sp(42.0,-33.0)","sp(41.75,-33.0)","sp(41.5,-33.0)","sp(41.25,-33.0)","sp(41.0,-33.0)","sp(40.75,-33.0)",...,"t2m(35.0,-23.0)","t2m(34.75,-23.0)","t2m(34.5,-23.0)","t2m(34.25,-23.0)","t2m(34.0,-23.0)","t2m(33.75,-23.0)","t2m(33.5,-23.0)","t2m(33.25,-23.0)","t2m(33.0,-23.0)",Cluster
0,0.769017,0.763551,0.758399,0.754214,0.749653,0.744597,0.739195,0.733298,0.726633,0.719377,...,0.312956,0.318721,0.328563,0.334153,0.337154,0.340863,0.343122,0.338263,0.331969,7
1,0.550508,0.539280,0.528615,0.520168,0.510767,0.500749,0.491037,0.481886,0.472325,0.462340,...,0.364151,0.363580,0.368759,0.371705,0.372464,0.370914,0.367119,0.358756,0.353140,9
2,0.406872,0.391876,0.377439,0.365852,0.353486,0.340685,0.327491,0.314235,0.300378,0.286403,...,0.384572,0.381714,0.380983,0.379379,0.377956,0.377146,0.375601,0.372759,0.375038,9
3,0.305465,0.282306,0.260121,0.241803,0.223679,0.206865,0.191111,0.176584,0.162586,0.149556,...,0.383770,0.374375,0.375442,0.382104,0.386662,0.388110,0.389274,0.388262,0.391018,9
4,0.421539,0.398652,0.376436,0.357109,0.337143,0.317032,0.296732,0.276819,0.256749,0.235928,...,0.371150,0.384408,0.382828,0.380475,0.379120,0.382806,0.388641,0.391018,0.390756,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,0.052561,0.047201,0.044078,0.046307,0.048426,0.051497,0.055427,0.060420,0.066650,0.073432,...,0.539179,0.545987,0.558745,0.566072,0.565328,0.562305,0.556372,0.547556,0.544027,1
361,0.000000,0.000000,0.001187,0.007059,0.011975,0.016685,0.020693,0.024733,0.028452,0.031877,...,0.529579,0.530462,0.536012,0.536755,0.532624,0.527203,0.521547,0.513866,0.509431,1
362,0.062689,0.061612,0.062288,0.067914,0.073401,0.079448,0.085514,0.091616,0.097813,0.104082,...,0.543597,0.551860,0.563950,0.569878,0.569516,0.566563,0.562419,0.556039,0.552496,24
363,0.013697,0.005919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.507099,0.516941,0.534650,0.546183,0.551323,0.552699,0.551484,0.549971,0.547362,24


In [ ]:
# Non-normalized
def handle_missing_values(input):
  var_mean = {}
  for i in input.data_vars:
    if input[i].isnull().sum().item() > 0:
      print('Null values in',i,'have been handled.')
      var_mean[str(i) + '_mean'] = input[i].mean().item()
      input[i] = input[i].fillna(var_mean[str(i) + '_mean'])
  return input

In [ ]:
# Non-normalized
# Function that creates a dictionary that holds the values of dates in each cluster
def get_datewise_clusters(formed_clusters): # classification
  Dates_Cluster = {}
  for i in set(formed_clusters): # classification
    Dates_Cluster['Dates_Cluster'+str(i)] = trans_data.index[trans_data.Cluster == i].to_list()
  return Dates_Cluster

In [ ]:
# Non-normalized
# Function that creats a dictionary that holds all the clusters
def n_nor_get_clusters(input,formed_clusters): # classification
  com_arr = []
  Clusters = {}
  Dates_Cluster = get_datewise_clusters(formed_clusters)
  for i in set(formed_clusters):
    for j in Dates_Cluster['Dates_Cluster'+str(i)]:
      #print(j)
      #print(Dates_Cluster['Dates_Cluster'+str(i)])
      arr = np.array(input.isel(time=j).to_array()) # input is data
      com_arr.append(arr)
    Clusters['Cluster' + str(i)] = np.array(com_arr)
    com_arr = []
  return Clusters  

In [ ]:
# Function that creates a dictionary that holds all the cluster centers
def n_nor_get_cluster_centers(input,formed_clusters): #classification
  Cluster_Centers = {}
  Clusters = n_nor_get_clusters(input,formed_clusters)
  for i in set(formed_clusters):
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
  return Cluster_Centers

In [ ]:
#Intra-spatial correlation coefficient Calculation Function
import functools 

def n_nor_intra_sp_corr(input,formed_clusters):
  mylist = []
  intra_sp_corr = []
  Clusters = n_nor_get_clusters(input,formed_clusters)
  Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)
  
  for i in range(len(Clusters)):
    mylist = []
    for j in range(len(Clusters['Cluster' + str(i)])):
      corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
       #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
      mylist.append(corr_coeff)
      average_corr_coeff = sum(mylist) / len(mylist)
    intra_sp_corr.append(average_corr_coeff)
  return intra_sp_corr

In [ ]:
# #Intra-spatial correlation coefficient Calculation Function
# import functools 

# def sp_corr(input,formed_clusters):
#   avg_cluster = {}
#   Clusters = n_nor_get_clusters(input,formed_clusters)
#   mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
#   for i in range(len(Clusters)):
#     avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
#   for i in range(len(Clusters)):
#     for j in range(len(Clusters)):
#       if i == j:
#         a = n_nor_intra_sp_corr(input,formed_clusters)
#         mat[i].iloc[j] = round(a[i],2)
#       else:
#         corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
#         mat[i].iloc[j] = corr_coeff
#           #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
#     return mat

##Normalized - Spatial Correlation##

In [ ]:
# Normalized
# Function that creates two dictionaries that hold all the clusters and cluster centers
def nor_get_clusters_and_centers(input,formed_clusters):
  Clusters = {}
  Cluster_Centers = {}
  for i in set(formed_clusters):
    Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
  return Clusters,Cluster_Centers

In [ ]:
#Intra-spatial correlation coefficient Calculation Function

def nor_intra_sp_corr(input,formed_clusters):
  mylist = []
  intra_sp_corr = []
  Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
  
  for i in range(len(Clusters)):
    mylist = []
    for j in range(len(Clusters['Cluster' + str(i)])):
      corr_coeff = pearson_PM(Clusters['Cluster' + str(i)][j], Cluster_Centers['Cluster_Center' + str(i)])
       #print('i: {}, j: {}, corr_coeff:{}'.format(i, j, corr_coeff))
      mylist.append(corr_coeff)
      average_corr_coeff = sum(mylist) / len(mylist)
    intra_sp_corr.append(average_corr_coeff)
  return intra_sp_corr

In [ ]:
# Non-normalized Spatial Correlation Calculation
def sp_corr(input,formed_clusters,normalize=False):
  inter_sp_corr = []
  avg_cluster = {}

  if normalize == False:
    input = handle_missing_values(input)
    Clusters = n_nor_get_clusters(input,formed_clusters)
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_sp_corr(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
          mat[i].iloc[j] = corr_coeff
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
        

  else:
    trans_data = datatransformation(input)

    # Data Normalization
    trans_data = datanormalization(trans_data)

    # Adding class centers and cluster numbers as columns to the dataframe
    trans_data['Cluster'] = formed_clusters

    # Rearranging the columns in the dataframe
    trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
  
    Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

    # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
    #trans_data = trans_data.reset_index()

  
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_sp_corr(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          corr_coeff = pearson_PM(avg_cluster['avg_cluster' + str(i)], avg_cluster['avg_cluster' + str(j)])
          mat[i].iloc[j] = corr_coeff
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
        

  return mat

In [ ]:
sp_final1 = sp_corr(data, classification, True)
sp_final1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,NaN,0.241096,0.042181,0.005363,0.166525,0.255225,0.226502,0.419983,0.285046,0.541447,...,-0.063104,-0.142217,-0.126974,0.229796,-0.075385,0.300298,0.515002,0.281781,-0.05364,0.012148
1,0.241096,NaN,0.432551,0.302063,0.924677,0.855959,0.747883,0.803418,0.490408,0.871019,...,-0.428764,-0.675802,-0.468656,-0.444057,-0.088166,0.174217,0.015036,-0.369894,0.401605,0.75898
2,0.042181,0.432551,NaN,0.75359,0.697439,0.661168,0.418909,0.291642,0.329479,0.183199,...,-0.530399,-0.380132,-0.575734,-0.777549,-0.26896,-0.499808,-0.438254,-0.455685,0.409216,0.63459
3,0.005363,0.302063,0.75359,NaN,0.441346,0.678898,0.584809,0.355713,0.618845,0.095412,...,-0.183047,-0.135703,-0.444967,-0.63028,-0.42139,-0.547117,-0.555778,-0.196461,0.350193,0.382592
4,0.166525,0.924677,0.697439,0.441346,NaN,0.860084,0.671622,0.678274,0.418651,0.716125,...,-0.593067,-0.693532,-0.601366,-0.658014,-0.149284,-0.077829,-0.156328,-0.501983,0.457324,0.839935
5,0.255225,0.855959,0.661168,0.678898,0.860084,NaN,0.859313,0.793303,0.698232,0.709478,...,-0.263208,-0.458498,-0.448418,-0.531249,-0.213057,-0.053561,-0.262302,-0.335124,0.319373,0.671188
6,0.226502,0.747883,0.418909,0.584809,0.671622,0.859313,NaN,0.936649,0.926273,0.638384,...,-0.054458,-0.506374,-0.469675,-0.280197,-0.518599,-0.028656,-0.242731,0.028972,0.108983,0.311386
7,0.419983,0.803418,0.291642,0.355713,0.678274,0.793303,0.936649,NaN,0.85067,0.804801,...,-0.12707,-0.61354,-0.479881,-0.140355,-0.455132,0.173343,0.033347,0.123456,0.059712,0.291446
8,0.285046,0.490408,0.329479,0.618845,0.418651,0.698232,0.926273,0.85067,NaN,0.457655,...,0.086701,-0.363264,-0.446155,-0.139012,-0.663022,-0.124008,-0.22162,0.290167,-0.022294,0.032882
9,0.541447,0.871019,0.183199,0.095412,0.716125,0.709478,0.638384,0.804801,0.457655,NaN,...,-0.300387,-0.556464,-0.313935,-0.160091,0.016013,0.426521,0.4052,-0.179377,0.331337,0.557464


In [ ]:
sp_final2 = sp_corr(data, classification, False)
sp_final2

Null values in sst have been handled.


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.0,0.776388,0.963863,0.920189,0.892985,0.948047,0.716524,0.879894,0.870708,0.903941,...,0.769828,0.924111,0.870468,0.903804,0.9005,0.891843,0.916039,0.895433,0.804819,0.904962
1,0.776388,0.91,0.81435,0.864379,0.879087,0.886628,0.930596,0.952292,0.897398,0.92884,...,0.903368,0.885284,0.94885,0.901633,0.94384,0.933294,0.88631,0.864634,0.855544,0.922154
2,0.963863,0.81435,0.99,0.931807,0.956135,0.945988,0.737256,0.89871,0.87758,0.927701,...,0.777417,0.93444,0.895551,0.899673,0.944153,0.906186,0.933915,0.891634,0.851209,0.946696
3,0.920189,0.864379,0.931807,0.99,0.866931,0.924546,0.890492,0.964077,0.981241,0.939461,...,0.930066,0.983202,0.959321,0.979408,0.959166,0.974497,0.973802,0.985535,0.946415,0.97505
4,0.892985,0.879087,0.956135,0.866931,0.96,0.927302,0.745523,0.886567,0.82332,0.912249,...,0.738586,0.876307,0.877187,0.847838,0.944059,0.875189,0.87915,0.82367,0.802135,0.92572
5,0.948047,0.886628,0.945988,0.924546,0.927302,0.92,0.822939,0.942592,0.901619,0.959201,...,0.83266,0.925469,0.917941,0.924771,0.951137,0.936612,0.92368,0.904431,0.801242,0.930679
6,0.716524,0.930596,0.737256,0.890492,0.745523,0.822939,0.93,0.952724,0.93754,0.883672,...,0.974457,0.874763,0.939657,0.918895,0.899714,0.935761,0.870756,0.900742,0.894649,0.889168
7,0.879894,0.952292,0.89871,0.964077,0.886567,0.942592,0.952724,0.94,0.97692,0.974547,...,0.956868,0.961013,0.987016,0.976473,0.980896,0.988884,0.96043,0.956525,0.92614,0.971851
8,0.870708,0.897398,0.87758,0.981241,0.82332,0.901619,0.93754,0.97692,0.96,0.948368,...,0.970636,0.97246,0.9738,0.982685,0.953063,0.982465,0.974245,0.983695,0.95808,0.96357
9,0.903941,0.92884,0.927701,0.939461,0.912249,0.959201,0.883672,0.974547,0.948368,0.91,...,0.898324,0.951853,0.962488,0.947536,0.975757,0.962379,0.969192,0.918892,0.887794,0.960602


#**Evaluation - RMSE** 

### RMSE - Non-normalized functions





In [ ]:
# Non-normalized
# Function that creates a dictionary that holds the values of dates in each cluster
def get_datewise_clusters(formed_clusters): # classification
  Dates_Cluster = {}
  for i in set(formed_clusters): # classification
    Dates_Cluster['Dates_Cluster'+str(i)] = trans_data.index[trans_data.Cluster == i].to_list()
  return Dates_Cluster

In [ ]:
# Non-normalized
# Function that creats a dictionary that holds all the clusters
def n_nor_get_clusters(input,formed_clusters): # classification
  com_arr = []
  Clusters = {}
  Dates_Cluster = get_datewise_clusters(formed_clusters)
  for i in set(formed_clusters):
    for j in Dates_Cluster['Dates_Cluster'+str(i)]:
      arr = np.array(input.isel(time=j).to_array()) # input is data
      com_arr.append(arr)
    Clusters['Cluster' + str(i)] = np.array(com_arr)
    com_arr = []
  return Clusters    

In [ ]:
# Non-normalized
# Function that creates a dictionary that holds all the cluster centers
def n_nor_get_cluster_centers(input,formed_clusters): #classification
  Cluster_Centers = {}
  Clusters = n_nor_get_clusters(input,formed_clusters)
  for i in set(formed_clusters):
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
  return Cluster_Centers

In [ ]:
# Non-normalized 
#Intra RMSE Calculation Function
def n_nor_intra_rmse(input,formed_clusters):
  sq_diff = []
  intra_rmse = []
  Clusters = n_nor_get_clusters(input,formed_clusters)
  Cluster_Centers = n_nor_get_cluster_centers(input,formed_clusters)

  for i in range(len(Clusters)):
    for j in range(len(Clusters['Cluster' + str(i)])):
      diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
      Sq_diff = (diff**2)
      sq_diff.append(Sq_diff)
    Sq_diff_sum = sum(sq_diff)
    sq_diff = []
    n = len(Clusters['Cluster' + str(i)])
    Sqrt_diff_sum = np.sqrt(sum(sum(sum(Sq_diff_sum/n))))
    intra_rmse.append(Sqrt_diff_sum)
  return intra_rmse

In [ ]:
# Non-normalized
def handle_missing_values(input):
  var_mean = {}
  for i in input.data_vars:
    if input[i].isnull().sum().item() > 0:
      print(i,'has null values')
      var_mean[str(i) + '_mean'] = input[i].mean().item()
      input[i] = input[i].fillna(var_mean[str(i) + '_mean'])
  return input

### RMSE - Normalized functions

In [ ]:
# Normalized
# Function that creates two dictionaries that hold all the clusters and cluster centers
def nor_get_clusters_and_centers(input,formed_clusters):
  Clusters = {}
  Cluster_Centers = {}
  for i in set(formed_clusters):
    Clusters['Cluster' + str(i)] = np.array(input[input.Cluster == i].drop(columns=['Cluster']))
    Cluster_Centers['Cluster_Center' + str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
  return Clusters,Cluster_Centers

In [ ]:
# Normalized
# Intra RMSE Calculation Function
def nor_intra_rmse(input,formed_clusters):
  intra_rmse = []
  sq_diff = []
  Clusters,Cluster_Centers = nor_get_clusters_and_centers(input,formed_clusters)
  for i in range(len(Clusters)):
    for j in range(len(Clusters['Cluster' + str(i)])):
      diff = Clusters['Cluster' + str(i)][j] - Cluster_Centers['Cluster_Center' + str(i)]
      Sq_diff = (diff**2)
      sq_diff.append(Sq_diff)
    Sq_diff_sum = sum(sum(sq_diff))
    sq_diff = []
    n = len(Clusters['Cluster' + str(i)])
    Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
    intra_rmse.append(Sqrt_diff_sum)
  return intra_rmse

In [ ]:
# RMSE Calculation
def RMSE(input,formed_clusters,normalize=False):
  inter_rmse = []
  avg_cluster = {}

  if normalize == False:
    input = handle_missing_values(input)
    Clusters = n_nor_get_clusters(input,formed_clusters)
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = n_nor_intra_rmse(input,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
          Sq_diff = (diff**2)
          #Sq_diff_sum = sum(Sq_diff)
          Sq_diff_sum = sum(sum(sum(Sq_diff)))
          #inter_rmse.append(np.sqrt(Sq_diff_sum))
          n = len(avg_cluster['avg_cluster'+str(i)][0])
          Sqrt_diff_sum = np.sqrt(Sq_diff_sum/n)
          mat[i].iloc[j] = round(Sqrt_diff_sum,2)
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())
        

  else:
    trans_data = datatransformation(input)

    # Data Normalization
    trans_data = datanormalization(trans_data)

    # Adding class centers and cluster numbers as columns to the dataframe
    trans_data['Cluster'] = formed_clusters

    # Rearranging the columns in the dataframe
    trans_data = trans_data[['Cluster'] + [c for c in trans_data if c not in ['Cluster']]]
  
    Clusters, Cluster_Centers = nor_get_clusters_and_centers(trans_data,formed_clusters)

    # Doing the below step after finding the cluster centers. Otherwise, we'll be calculating mean on date (index) too.
    #trans_data = trans_data.reset_index()

  
    mat = pd.DataFrame(columns=range(len(Clusters)),index=range(len(Clusters)))
    for i in range(len(Clusters)):
      avg_cluster['avg_cluster'+str(i)] = np.mean(Clusters['Cluster' + str(i)],axis=0)
    for i in range(len(Clusters)):
      for j in range(len(Clusters)):
        if i == j:
          a = nor_intra_rmse(trans_data,formed_clusters)
          mat[i].iloc[j] = round(a[i],2)
        else:
          diff = avg_cluster['avg_cluster' + str(i)] - avg_cluster['avg_cluster' + str(j)]
          Sq_diff = (diff**2)
          #Sq_diff_sum = sum(Sq_diff)
          Sq_diff_sum = sum(Sq_diff)
          #inter_rmse.append(np.sqrt(Sq_diff_sum))
          Sqrt_diff_sum = np.sqrt(Sq_diff_sum)
          mat[i].iloc[j] = round(Sqrt_diff_sum,2)
          #print('Inter RMSE between cluster',i,'and cluster',j,'is:',inter_rmse.pop())

  return mat

In [ ]:
final1 = RMSE(data,classification,True)
final1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,46.94,36.64,33.05,43.8,38.63,44.66,37.15,32.21,31.07,...,53.73,49.53,55.97,47.56,50.49,45.8,31.87,33.01,33.41,41.72
1,46.94,9.77,34.32,41.23,12.55,16.98,21.08,18.77,32.45,19.86,...,38.9,44.07,38.39,36.57,31.45,26.62,31.16,43.67,32.48,19.88
2,36.64,34.32,12.08,19.56,24.94,23.19,33.48,32.44,29.34,32.98,...,48.5,44.99,49.29,46.89,42.17,40.07,35.53,41.62,25.13,23.93
3,33.05,41.23,19.56,13.29,35.23,26.9,33.79,34.05,23.04,35.76,...,49.73,45.48,53.03,49.4,47.92,44.75,38.26,38.17,26.37,32.83
4,43.8,12.55,24.94,35.23,8.75,14.61,23.5,21.69,31.32,21.73,...,44.25,46.45,43.69,41.84,35.54,32.04,33.08,44.88,28.9,15.37
5,38.63,16.98,23.19,26.9,14.61,11.03,15.88,16.14,21.51,19.55,...,39.15,41.19,41.11,38.72,34.96,30.42,31.1,39.13,27.79,19.68
6,44.66,21.08,33.48,33.79,23.5,15.88,8.17,11.34,18.14,25.44,...,38.83,45.64,43.74,39.16,40.96,33.24,36.04,38.44,36.55,32.13
7,37.15,18.77,32.44,34.05,21.69,16.14,11.34,11.08,17.2,16.72,...,37.91,43.8,41.81,34.98,37.74,29.11,28.51,32.67,32.69,29.41
8,32.21,32.45,29.34,23.04,31.32,21.51,18.14,17.2,12.2,26.3,...,41.03,43.64,47.08,39.76,44.22,36.6,32.08,28.39,31.47,35.32
9,31.07,19.86,32.98,35.76,21.73,19.55,25.44,16.72,26.3,13.43,...,42.43,44.28,43.0,37.41,35.68,29.84,23.54,36.59,26.42,23.67


In [ ]:
#final1.to_csv("/content/drive/MyDrive/Private/Image_Similarity/PCA_daily_final1.csv")

In [ ]:
#df1 = pd.read_csv("/content/drive/MyDrive/Private/Image_Similarity/PCA_daily_final1.csv")

In [ ]:
#df1 = df1.iloc[:,1:]

In [ ]:
final2 = RMSE(data,classification,False)
final2

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,5270962.87,1550525.51,2237390.68,3764798.19,4067304.1,5192199.42,4543354.01,3392819.17,4347165.1,...,4748971.19,2996523.73,4625915.35,3682324.1,4131838.05,4190543.51,2789785.12,2609136.8,3438598.28,3344788.78
1,5270962.87,4755352.06,4744423.24,4735913.72,1932441.67,1322550.75,587214.82,854949.46,2582968.8,1065558.38,...,1060930.55,2713468.76,799232.84,1909182.75,1281893.77,1269728.46,3122023.95,5094686.87,3845387.91,2318682.63
2,1550525.51,4744423.24,7264941.77,1953174.07,3019999.14,3604784.08,4690061.08,4029705.41,2944879.05,3813813.43,...,4271825.18,2503112.68,4099623.89,3253598.98,3522583.3,3682802.28,2251093.25,2588002.53,2793213.21,2660258.91
3,2237390.68,4735913.72,1953174.07,7453658.88,3498639.87,3737481.5,4484828.35,3929310.37,2287197.73,3844150.92,...,3941506.13,2173268.56,4011646.96,2931815.29,3535177.0,3519503.7,1909351.14,1017426.11,1845175.65,2535936.08
4,3764798.19,1932441.67,3019999.14,3498639.87,6103539.62,1166634.18,2071100.14,1497941.09,2003728.3,1319600.77,...,1955012.13,1795479.98,1565579.24,1576870.8,1029653.93,1410882.66,2073959.29,4012363.5,2984114.34,1282680.79
5,4067304.1,1322550.75,3604784.08,3737481.5,1166634.18,6907251.61,1351177.66,777554.5,1825504.9,636018.02,...,1246862.11,1825347.14,920037.08,1178646.52,714986.04,801673.63,2211529.07,4156132.81,3259292.56,1510627.07
6,5192199.42,587214.82,4690061.08,4484828.35,2071100.14,1351177.66,4984250.85,689407.85,2295041.8,1049798.4,...,664681.37,2542134.3,690366.13,1681038.62,1225223.83,1081641.0,2953180.93,4826848.43,3575906.6,2190008.31
7,4543354.01,854949.46,4029705.41,3929310.37,1497941.09,777554.5,689407.85,5831529.38,1787497.07,473561.39,...,582098.75,1954638.87,309799.86,1131764.36,605858.31,510561.35,2361827.24,4310910.46,3130263.22,1592133.03
8,3392819.17,2582968.8,2944879.05,2287197.73,2003728.3,1825504.9,2295041.8,1787497.07,9358952.06,1753823.41,...,1734462.78,824914.51,1867364.0,901210.01,1522300.03,1397621.05,983049.04,2624160.37,1666611.89,928012.52
9,4347165.1,1065558.38,3813813.43,3844150.92,1319600.77,636018.02,1049798.4,473561.39,1753823.41,7951496.34,...,916876.93,1848666.69,582316.54,1150940.4,554999.75,637344.71,2183736.33,4261849.16,3107801.62,1499090.72


#**Evaluation - Silhouette Coefficient**

In [ ]:
def silhouette_score1(X, labels, pass_trans_data = True, *, metric="euclidean", sample_size=None, random_state=None, **kwds):
   if pass_trans_data == True:
          k = list(locals().values()) # k will be the list which holds the values of the parameters that are being passed to the function
          #path = str(k[0]) # path will hold the first parameter's value
          #fullpath = os.path.join("/content/drive/MyDrive/Private/Image_Similarity/Transformed_data/" + path + ".csv")
          

          path = ("/content/drive/MyDrive/Private/Image_Similarity/AFtrans_data_daily_PCA_Transformed.csv")
          #X1 = pd.read_csv(fullpath, index_col=[0]) # saved transformed dataframe will be read
          X1 = pd.read_csv(path, index_col=[0])
          X1 = datanormalization(X1) # the data will be normalized

          if sample_size is not None:
            X1, labels = check_X_y(X1, labels, accept_sparse=["csc", "csr"])
            random_state = check_random_state(random_state)
            indices = random_state.permutation(X1.shape[0])[:sample_size]
            if metric == "precomputed":
              X1, labels = X1[indices].T[indices].T, labels[indices]
            else:
              X1, labels = X1[indices], labels[indices]
          return np.mean(silhouette_samples(X1, labels, metric=metric, **kwds))
   
   else:
     X1 = data[X]
     X1 = datatransformation(X)
     X1 = datanormalization(X1)   
     if sample_size is not None:
        X1, labels = check_X_y(X1, labels, accept_sparse=["csc", "csr"])
        random_state = check_random_state(random_state)
        indices = random_state.permutation(X1.shape[0])[:sample_size]
        if metric == "precomputed":
            X1, labels = X1[indices].T[indices].T, labels[indices]
        else:
            X1, labels = X1[indices], labels[indices]
     return np.mean(silhouette_samples(X1, labels, metric=metric, **kwds))

In [ ]:
silhouette_Coefficient = silhouette_score1('data', classification)
print("The average silhouette_score is :", silhouette_Coefficient)

FileNotFoundError: ignored